In [93]:
"""
This is a modified version of the Monte Carlo March Madness Bracket code originally written by Michael Lerner, PhD. Original code and more detail can be found here:
http://www.mglerner.com/blog/?p=16
"""

'\nThis is a modified version of the Monte Carlo March Madness Bracket code originally written by Michael Lerner, PhD. Original code and more detail can be found here:\nhttp://www.mglerner.com/blog/?p=16\n'

In [94]:
import os, os.path
import _pickle
import numpy
import pylab
import scipy
import pandas
from collections import Counter, OrderedDict
from copy import deepcopy
from functools import wraps
from hashlib import md5
from itertools import chain, zip_longest
from numpy.random import random, randint
from numpy import exp,array,zeros,inf
from random import choice,shuffle
from time import sleep, time

In [95]:
def grouper(n, iterable, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [96]:
def pairs(iterable):
    return grouper(2,iterable)

In [97]:
def playgame(team1, team2, T):
    """There's a difference between flipping a game in an existing
    bracket, and playing a game from scratch. If we're going to just
    use Boltzmann statistics to play a game from scratch, we can make
    life easy by using the Boltzmann factor to directly pick a
    winner.

    """
    ediff = energy_of_flipping(team1, team2)
    boltzmann_factor = exp(-ediff/T)

    win_prob = boltzmann_factor/(1+boltzmann_factor) if boltzmann_factor < inf else 1
    # So, prob of team 1 winning is then boltzmann_factor/(1+boltzmann_factor)
    if random() >= win_prob:
        return (team1,team2)
    else:
        return (team2,team1)

In [98]:
def playgamesfortesting(team1, team2, ntrials, T):
    print("Boltzmann tells that the ratio of team1 winning to team 2"+ 
          "winning should be")
    print(exp(-energy_of_flipping(team1,team2)/T))
    wins = {team1:0,team2:0}
    for i in range(ntrials):
        winner,loser = playgame(team1,team2,T)
        wins[winner] = wins[winner] + 1
    print("wins {} {} {} {} {}".format(wins, wins[team1]/wins[team2], 
                                       wins[team2]/wins[team1], 
                                       wins[team1]/ntrials, 
                                       wins[team2]/ntrials))

In [99]:
def playround(teams, T):
    winners = []
    losers = []
    for (team1, team2) in pairs(teams):
        winner, loser = playgame(team1,team2,T)
        winners.append(winner)
        losers.append(loser)
    return winners,losers

In [100]:
def runbracket(teams, T):
    # How many rounds do we need?
    nrounds = int(numpy.log2(len(teams)))
    winners = teams #they won to get here!
    all_winners = [winners]
    for round in range(nrounds):
        winners, losers = playround(winners, T)
        all_winners.append(winners)
    return all_winners

In [101]:
def bracket_energy(all_winners):
    total_energy = 0.0
    for i in range(len(all_winners)-1):
        games = pairs(all_winners[i])
        winners = all_winners[i+1]
        for (team1, team2),winner in zip(games, winners):
            if winner == team1:
                total_energy += energy_game(team1, team2)
            else:
                total_energy += energy_game(team2, team1)
    return total_energy

In [102]:
def getroundmap(bracket, include_game_number):
    games_in_rounds = [2**i for i in reversed(range(len(bracket)-1))]
    round = {}
    g = 0
    for (i,gir) in enumerate(games_in_rounds):
        for j in range(gir):
            if include_game_number:
                round[g] = (i,j)
            else:
                round[g] = i
            g += 1
    return round

In [103]:
class Bracket(object):
    def __init__(self, teams, T, bracket=None):
        """
        
        Arguments:
        - `teams`:
        - `T`:
        """
        self.teams = teams
        self.T = T
        if bracket is None:
            self.bracket = runbracket(self.teams, self.T)
        else:
            self.bracket = bracket
        self.games_in_rounds = [2**i for i in 
                                reversed(range(len(self.bracket)-1))]
        self.roundmap = getroundmap(self.bracket, include_game_number=False)
        self.roundmap_with_game_numbers = getroundmap(self.bracket, 
                                                      include_game_number=True)
    def copy(self):
        return self.__class__(self.teams, self.T,  
                              bracket=[l[:] for l in self.bracket])
    def energy(self):
        return bracket_energy(self.bracket)
    def __str__(self):
        return bracket_to_string(self.bracket)
    __repr__ = __str__
    def __hash__(self):
        return hash(tuple([tuple(aw) for aw in self.bracket]))
    def game(self, g):
        """Return (team1,team2,winner).
        """
        t1, t2, win = self._getgameidxs(g)
        return (self.bracket[t1[0]][t1[1]], self.bracket[t2[0]][t2[1]],
                self.bracket[win[0]][win[1]])

    def _round_teaminround_to_game(self,r,gir):
        return sum(self.games_in_rounds[:r]) + int(gir/2)

    def _getgameidxs(self, g):
        # we'll return (round,game) for each of team1, team2, winner
        # 0 1 2 3 4 5 6 7 # teams 1
        # 0 0 1 1 2 2 3 3 # games 1
        # 0 2 4 6 # teams 2
        # 0 0 1 1 # games 2
        round, game_in_round = self.roundmap_with_game_numbers[g]
        return ((round,2*game_in_round), (round,2*game_in_round+1), 
                (round+1,game_in_round))
    def _setwinner(self, g, winner):
        """ JUST SETS THE WINNER, DOES NOT LOOK TO NEXT ROUND! USE SWAP FOR 
        THAT! 
        """
        t1,t2,win = self._getgameidxs(g)
        self.bracket[win[0]][win[1]] = winner
    def swap(self, g):
        """
        NOTE: This does not check 
        """
        team1, team2, winner = self.game(g)
        if team1 == winner:
            self._setwinner(g, team2)
        else:
            self._setwinner(g, team1)
        wr, wt = self._getgameidxs(g)[2]
        ng = self._round_teaminround_to_game(wr, wt)
        while ng < sum(self.games_in_rounds):
            #print "Now need to check game",wr,wt,ng,self.game(ng)
            winner, loser = playgame(self.game(ng)[0], self.game(ng)[1], self.T)
            self._setwinner(ng, winner)
            wr, wt = self._getgameidxs(ng)[2]
            ng = self._round_teaminround_to_game(wr, wt)
    def upsets(self):
        result = 0
        for g in range(sum(self.games_in_rounds)):
            t1,t2,win = self.game(g)
            if t1 == win:
                los = t2
            else:
                los = t1
            if energy_of_flipping(win,los) < 0:
                result += 1
        return result




In [104]:
def print_bracket(bracket):
    print(bracket_to_string(bracket))

In [105]:
class memoized(object):
    """Decorator that caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned, and
    not re-evaluated.
    """
    def __init__(self, func):
        self.func = func
        self.cache = {}
    def __call__(self, *args):
        try:
            return self.cache[args]
        except KeyError:
            value = self.func(*args)
            self.cache[args] = value
            return value
        except TypeError:
            # uncachable -- for instance, passing a list as an argument.
            # Better to not cache than to blow up entirely.
            return self.func(*args)
    def __repr__(self):
        """Return the function's docstring."""
        return self.func.__doc__
    def __get__(self, obj, objtype):
        """Support instance methods."""
        fn = functools.partial(self.__call__, obj)
        fn.reset = self._reset
        fn.getcache = self._get_cache
        fn.setcache = self._set_cache
        fn.cachedict = self._get_cache_dict
        return fn
        
    def _reset(self):
        self.cache = {}
    def _get_cache(self,key):
        return self.cache[key]
    def _set_cache(self,key,value):
        self.cache[key] = value
    def _get_cache_dict(self):
        return self.cache

In [106]:
def gather_uniquestats(brackets):
    lb = Bracket(brackets[0].teams, T=0.0000001) # low bracket
    low_hash = hash(lb)
    cnt = Counter()
    unique_brackets = []
    lowest_sightings = []
    brackets_by_hash = {}
    for b in brackets:
        h = hash(b)
        cnt[h] += 1
        unique_brackets.append(len(cnt))
        lowest_sightings.append(cnt[low_hash])
        brackets_by_hash[h] = b
    h,c = cnt.most_common(1)[0]
    mcb = brackets_by_hash[h] # most comon bracket
    return lb, mcb, c, unique_brackets, lowest_sightings

In [107]:
# Could be St. Mary's instead of Middle Tennessee
# Could be Liberty instead of North Carolina A&T.
teams = {}
teams['midwest'] = ['Louisville','North Carolina A&T','Colorado St.','Missouri',
                    'Oklahoma St.','Oregon','St. Louis','New Mexico St.',
                    'Memphis',"St. Mary's",'Michigan St.','Valparaiso',
                    'Creighton','Cincinnati','Duke','Albany']

#Could be La Salle instead of Boise St.
teams['west'] = ['Gonzaga','Southern','Pittsburgh','Wichita St.',
                 'Wisconsin','Mississippi','Kansas St.','La Salle',
                 'Arizona','Belmont','New Mexico','Harvard',
                 'Notre Dame','Iowa St.','Ohio St.','Iona']

teams['south'] = ['Kansas','Western Kentucky','North Carolina','Villanova',
                  'Virginia Commonwealth','Akron','Michigan','South Dakota St.',
                  'UCLA','Minnesota','Florida','Northwestern St.',
                  'San Diego St.','Oklahoma','Georgetown','Florida Gulf Coast']

# Could be Long Island instead of James Madison
teams['east'] = ['Indiana','James Madison','North Carolina St.','Temple',
                 'Nevada Las Vegas','California','Syracuse','Montana',
                 'Butler','Bucknell','Marquette','Davidson',
                 'Illinois','Colorado','Miami FL','Pacific']

#df = pandas.DataFrame(teams);
#df.to_csv('teams.csv')

teams = pandas.read_csv('teams.csv');

# These are all listed in the same order:
_rankings = [1,16,8,9,5,12,4,13,6,11,3,14,7,10,2,15]
regional_rankings = {}
for region in teams:
    for (team,rank) in zip(teams[region],_rankings):
        regional_rankings[team] = rank + random()/10

regions = {}
for region in teams:
    for team in teams[region]:
        regions[team] = region
all_teams = teams['midwest'] + teams['south'] + teams['west'] + teams['east']

kenpom = pandas.read_csv('pomeroy.csv')
#print(kenpom);

sagarin = pandas.read_csv('sagarin.csv');
#print(sagarin);

In [108]:
#strength = kenpom['Luck']
strength = sagarin

#T = 0.5 # In units of epsilon/k
#T = 2.5 # In units of epsilon/k

#@memoized
def energy_game(winner, loser):
    """This is where you'll input your own energy functions. Here are
    some of the things we talked about in class. Remember that you
    want the energy of an "expected" outcome to be lower than that of
    an upset.

    """
    #result = -(strength[winner] - strength[loser])
    #result = regional_rankings[winner] - regional_rankings[loser]
    #result = regional_rankings[winner]/regional_rankings[loser]
    #result = -(strength[winner]/strength[loser])
    #result = random()
    #result = color of team 1 jersey better than color of team 2 jersey
    #print "energy_game(",winner,loser,")",result
    if(sagarin.loc[sagarin['Team'] == winner].empty):
        print(winner)
        wrating = 10
    else:
        wrating = sagarin.loc[sagarin['Team'] == winner]['Rating'].values[0].astype(float)
        
    if(sagarin.loc[sagarin['Team'] == loser].empty):
        print(loser)
        lrating = 10
    else:
        lrating = sagarin.loc[sagarin['Team'] == loser]['Rating'].values[0].astype(float)
    result = -(wrating - lrating)
    return result

#@memoized
def energy_of_flipping(current_winner, current_loser):
    """Given the current winner and the current loser, this calculates
    the energy of swapping, i.e. having the current winner lose.
    """
    return (energy_game(current_loser, current_winner) - 
            energy_game(current_winner, current_loser))

#@profile
def simulate(ntrials, region, T, printonswap=False, showvis=True, newfig=False,
             teamdesc=None, printbrackets=True):
    """
    If region is "west" "midwest" "south" or "east" we'll run a bracket based 
    just on those teams.
    If it's "all" we'll run a full bracket.
    If it's a list of teams, we'll run a bracket based just on that list.

    So, one way you might want to do things is to simulate 10000 runs for each 
    of the four brackets,
    then run your final four explicitly, e.g.

    T = 1.5
    simulate(10000,'midwest',T)
    # record results
    simulate(10000,'south',T)
    # record results
    simulate(10000,'west',T)
    # record results
    simulate(10000,'east',T)
    # record results

    simulate(10000,['Louisville','Kansas','Wisconsin','Indiana'],T)
    """

    if type(region)  in (type([]), type(())):
        teams = region[:]
    else:
        teams = teams[region]
    b = Bracket(teams, T)
    energy = b.energy()
    ng = sum(b.games_in_rounds) # total number of games
    # Let's collect some statistics
    brackets = []
    for trial in range(ntrials):
        g = randint(0, ng) # choose a random game to swap
        #print "attempted swap for game",g#,"in round",round[g]
        #newbracket = deepcopy(b)
        newbracket = b.copy()
        newbracket.swap(g)
        newenergy = newbracket.energy()
        ediff = newenergy - energy
        if ediff <= 0:
            b = newbracket
            energy = newenergy
            if printonswap:
                print("LOWER")
                print(b)
        else:
            if random() < exp(-ediff/T):
                b = newbracket
                energy = newenergy
                if printonswap:
                    print("HIGHER")
                    print(b)
        brackets.append(b)


    lb, mcb, mcb_count, unique_brackets, lowest_sightings = \
        gather_uniquestats(brackets)
    if showvis:
        showstats(brackets, unique_brackets, lowest_sightings, 
                                newfig=newfig, teamdesc=teamdesc)
    if printbrackets:
        print("Lowest energy bracket")
        print(lb)
        print("Most common bracket (%s)"%mcb_count)
        print(mcb)
    return (lb,mcb)

def runbracket1(ntrials, T):
    simulate(ntrials,'all',T)

def runbracket2(ntrials1, ntrials2, T):
    results = {}
    regions = 'midwest west south east'.split()
    for (i,region) in enumerate(regions):
        results[region] = simulate(ntrials1, region, T, newfig=i, 
                                   teamdesc=region, printbrackets=False)
    # Make a new bracket from our final four
    teams = [results[region][1].bracket[-1][0] for region in regions]
    ff_lb, ff_mcb = simulate(ntrials2, teams, T, newfig=i+1, 
                             teamdesc="Final Four", printbrackets=False)

    print("YOUR LOWEST ENERGY BRACKETS")
    for region in regions:
        print("LOWEST ENERGY BRACKET FOR REGION"), region
        print(results[region][0])
        print()
    print("LOWEST ENERGY BRACKET FOR FINAL FOUR")
    print(ff_lb) 
        
    print("YOUR MOST COMMON BRACKETS")
    for region in regions:
        print( "MOST COMMON BRACKET FOR REGION", region)
        print(results[region][1])
        print()
    print("MOST COMMON BRACKET FOR FINAL FOUR")
    print(ff_mcb)



In [109]:
def movingaverage(interval, window_size):
    window= ones(int(window_size))/float(window_size)
    return convolve(interval, window, 'same')

def plotone(brackets, label, subplot1, subplot2, values=None, label2=None, 
            values2=None, teamdesc=None, useavg=False):
    """
    Plotting too many points causes lots of trouble for matplotlib. At
    the moment, we deal with that by plotting at most 50000 points,
    skipping evenly through the data if needed.
    """
    maxpts = 50000

    ntrials = len(brackets)
    if values is None:
        try:
            values = [getattr(b,label)() for b in brackets]
        except TypeError:
            values = [getattr(b,label) for b in brackets]

    if len(values) >= 50000:
        step = divmod(len(values),maxpts)[0]
    else:
        step = 1
    
    pl.subplot(subplot1)
    pl.plot(xrange(0,ntrials,step),values[::step],'.',label=label)
    if useavg:
        # want something like 2000 windows
        if step > 1:
            npts = maxpts
        else:
            npts = len(values)
        avgstep = divmod(len(values),int(npts/25))[0]
        
        pl.plot(xrange(0,ntrials,step),movingaverage(values[::step],avgstep),'-',label='avg. '+label)
            
    pl.ylabel(label.capitalize())
    pl.xlabel('Game')
    if teamdesc is not None:
        pl.title('%s over the trajectory, T=%s, %s'%(label.capitalize(),
                                                     brackets[0].T,teamdesc))
    else:
        pl.title('%s over the trajectory, T=%s'%(label.capitalize(),
                                                 brackets[0].T))
    #pl.legend()
    pl.subplot(subplot2)
    if values2 is None:
        if ntrials > 1000:
            nbins = min(int(ntrials/100),200)
        else:
            nbins = 10
        pl.hist(values,bins=nbins)
        pl.title('%s distribution, T=%s'%(label.capitalize(), brackets[0].T))
    else:
        pl.subplot(subplot2)
        pl.plot(xrange(0,ntrials,step),values2[::step],'.',label=label2)
        pl.ylabel(label2.capitalize())
        pl.xlabel('Game')
        pl.title('%s over the trajectory, T=%s'%(label2.capitalize(),
                                                 brackets[0].T))

    
def showstats(brackets, unique_brackets, lowest_sightings, newfig=False,
              teamdesc=None):
    if newfig is not False:
        if newfig is True:
            pl.figure()
        else:
            pl.figure(newfig)
    pl.clf()
    plotone(brackets, 'energy', 231, 234, teamdesc=teamdesc)
    plotone(brackets, 'upsets', 232, 235, teamdesc=teamdesc)
    plotone(brackets, 'Unique brackets', 233, 236, values=unique_brackets,
            label2="Lowest Energy Sightings", values2=lowest_sightings,
            teamdesc=teamdesc)
    pl.show()



In [134]:

b = Bracket(teams=teams,T=0.5)

east
Unnamed: 0
Unnamed: 0
east
south
midwest
midwest
south
None
west
west
None
midwest
Unnamed: 0
Unnamed: 0
midwest
None
west
west
None
west
midwest
midwest
west
None
midwest
midwest
None


In [135]:
""" Cute version that prints out brackets for 2, 4, 8, 16, 32, 64, etc. """
result = ''
aw = b.bracket # save some typing later
nrounds = len(b.bracket) #int(np.log2(len(teams)))
for r in b.bracket:
    print(r)


    Unnamed: 0               east                midwest           south  \
0            0          Villanova                 Kansas        Virginia   
1            1            Radford           Pennsylvania            UMBC   
2            2      Virginia Tech             Seton Hall       Creighton   
3            3            Alabama     North Carolina St.      Kansas St.   
4            4      West Virginia                Clemson        Kentucky   
5            5         Murray St.         New Mexico St.        Davidson   
6            6        Wichita St.                 Auburn         Arizona   
7            7           Marshall  College of Charleston         Buffalo   
8            8            Florida                    TCU        Miami FL   
9            9               UCLA            Arizona St.  Loyola-Chicago   
10          10         Texas Tech           Michigan St.       Tennessee   
11          11  Stephen F. Austin               Bucknell      Wright St.   
12          